## Install Libraries

We will install the software libraries necessary for this week's tutorials, and test them to make sure they work.

You can try to install all  required componenets by running the comand below.

If that does not work you can slo use command-line or terminal. Have alook at the linked page for more detail. [the linked wiki page will help you](https://github.com/DavidGoedicke/RealtimeAudioClassification/wiki/Command-lines-and-terminals-a-quick-introduction.md)

The libraries that we will use.

[NumPy](https://en.wikipedia.org/wiki/NumPy) (Python support for large, multi-dimensional arrays and matrices ).

[MatplotLib](https://matplotlib.org) (Python 2D plotting library).

[pyTorch](https://pytorch.org/get-started/locally/) (Python Machine Learning Library).

[sklearn](https://scikit-learn.org/stable/) (SciKit Machine Learning library).

[librosa](https://librosa.github.io/librosa/) (Music and Audio analysis library).

[PyAudio](https://people.csail.mit.edu/hubert/pyaudio/) (Python bindings for PortAudio, the cross-platform audio I/O library).


In [ ]:
pip install numpy matplotlib torch torchvision scikit-learn librosa pyaudio numpy_ringbuffer ipywidgets

## Test setup

In the code below, we load ("import") the libraries that we just installed above, to make sure that Python can find the libraries. 


In [ ]:
import csv
import os
import wave
import time
import multiprocessing
from threading import Thread
import os
print("-------Found all built-in libraries.-------")
import numpy as np
print("Found numpy ",np.__version__)
import matplotlib
print("Found matplotlib ",matplotlib.__version__)
print("-------Found all math libraries.-------")
import pyaudio
print("Found pyaudio ",pyaudio.__version__)
import librosa
print("Found librosa ",librosa.__version__)
print("-------Found all audio libraries.-------")
import torch
print("Found torch ",torch.__version__)
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix
print("-------Found all machine learning libraries.-------")


If the above code runs without a problem, all libraries have installed and can be loaded. While it does not gurantee that everything will work, this is a very good sign.
